Copyright (c) 2023 Habana Labs, Ltd. an Intel Company.  
Copyright (c) 2017, Pytorch contributors All rights reserved.
##### BSD 3-Clause License
Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

# Model Migration from GPU to the Intel&reg; Gaudi&reg; 2 AI Accelerator 

The GPU Migration toolkit simplifies migrating PyTorch models that run on GPU-based architecture to run on the Intel® Gaudi® AI accelerator. Rather than manually replacing Python API calls that have dependencies on GPU libraries with Gaudi-specific API calls, the toolkit automates this process so you can run your model with fewer modifications. 

The GPU Migration toolkit maps specific API calls from the Python libraries and modules listed below to the appropriate equivalents in the Intel Gaudi software:

* torch.cuda  
* Torch API with GPU related parameters. For example, torch.randn(device=”cuda”)  

The toolkit does not optimize the performance of the model, so further modifications may be required. For more details, refer to Model Performance Optimization Guide.

In this notebook we will demonstrate how to use the GPU Migration toolset on a ResNet50 model which is based on open source implementation of ResNet50.  

Refer to the [GPU Migration Toolkit](https://docs.habana.ai/en/latest/PyTorch/PyTorch_Model_Porting/GPU_Migration_Toolkit/GPU_Migration_Toolkit.html) for more information.  

In addition to this ResNet50 migration, there is a GPU Migration example on the Intel Gaudi GitHub page [here](https://github.com/HabanaAI/Model-References/tree/master/PyTorch/examples/gpu_migration)

### Enabling the GPU Migration Toolkit Summary

#### Set the library
Import the habana_frameworks.torch.gpu_migration package at the beginning of the primary Python script (main.py, train.py, etc.):
`import habana_frameworks.torch.gpu_migration`
Alternatively, you can use PT_HPU_GPU_MIGRATION=1 environment variable when running the primary Python script (main.py, train.py, etc.): `PT_HPU_GPU_MIGRATION=1 $PYTHON main.py`

#### Set the Mark Step
Add mark_step(). In Lazy mode, mark_step() must be added in all training scripts right after loss.backward() and optimizer.step().

`htcore.mark_step()`   Note that if your model is using torch.compile, this step is not needed. 

#### Running Migrated code and logging changes 
Make sure that any device selection argument passed to the script is configured as if the script is running on a GPU. For example, add --cuda or --device gpu in the runtime command of your model. This will guarantee that the GPU Migration toolkit accurately detects and migrates instructions.

You can enable the logging feature, included in the GPU Migration toolkit, by setting the `GPU_MIGRATION_LOG_LEVEL` environment variable like this example:   
`GPU_MIGRATION_LOG_LEVEL=3 PT_HPU_GPU_MIGRATION=1 $PYTHON main.py`

#### For More Information
For more information regarding the use and configuration of the GPU Migration Toolkit and its limitations, please refer to the documentation here (https://docs.habana.ai/en/latest/PyTorch/PyTorch_Model_Porting/GPU_Migration_Toolkit/GPU_Migration_Toolkit.htm).

In [ ]:
# Start with the `exit()` command to restart the Python kernel to ensure that there are no other processes holding the Intel Gaudi Accelerator as you start to run this notebook.
exit()

#### Running the ResNet50 Example
The remainder of the notebook will show how the tool works with the [ResNet50 example](https://github.com/HabanaAI/Model-References/tree/master/PyTorch/examples/gpu_migration) from the GPU Migration examples in Model-References

In [ ]:
%cd ~/Gaudi-tutorials/PyTorch/Single_card_tutorials
!git clone -b 1.15.1 https://github.com/habanaai/Model-References

#### Navigate to the model example to begin the run

In [ ]:
%cd ~/Gaudi-tutorials/PyTorch/Single_card_tutorials/Model-References/PyTorch/examples/gpu_migration/computer_vision/classification/torchvision

#### Download dataset - OPTIONAL
To fully run this example you can download the ImageNet 2012 dataset.  It needs to be organized according to PyTorch requirements, and as specified in the scripts of [imagenet-multiGPU.torch](https://github.com/soumith/imagenet-multiGPU.torch).   You do NOT need to have the dataset loaded to see the Migration steps and logging.    

If you do not download the dataset the training will not complete the execution, but you will see the GPU Migration changes in the logfile.

#### Import GPU Migration Toolkit package and Habana Torch Library
Look into train.py, you will see in the first line that we will load the `gpu.migration` library which is already included in the Intel Gaudi Software: 

In [4]:
%%sh
cat -n train.py | head -n 21 | tail -n 18

     4	import habana_frameworks.torch.gpu_migration
     5	import datetime
     6	import os
     7	import time
     8	import warnings
     9	
    10	import presets
    11	import torch
    12	import torch.utils.data
    13	import torchvision
    14	import transforms
    15	import utils
    16	import habana_frameworks.torch.utils.experimental as htexp
    17	from sampler import RASampler
    18	from torch import nn
    19	from torch.utils.data.dataloader import default_collate
    20	from torchvision.transforms.functional import InterpolationMode


#### Placing mark_step()
You will have to place the mark_step() function after the optimizer and loss.backward calculations

In [5]:
%%sh
cat -n train.py | head -n 51 | tail -n 14

    38	        optimizer.zero_grad(set_to_none=True)
    39	        if scaler is not None:
    40	            scaler.scale(loss).backward()
    41	            htcore.mark_step()
    42	            if args.clip_grad_norm is not None:
    43	                # we should unscale the gradients of optimizer's assigned params if do gradient clipping
    44	                scaler.unscale_(optimizer)
    45	                nn.utils.clip_grad_norm_(model.parameters(), args.clip_grad_norm)
    46	            scaler.step(optimizer)
    47	            htcore.mark_step()
    48	            scaler.update()
    49	        else:
    50	            loss.backward()
    51	            htcore.mark_step()


#### Run the following command to start multi-HPU training.
We're now ready to run the training.  You will see that we've added the logging command at the beginning of the run: `GPU_MIGRATION_LOG_LEVEL=1` to show the output.   No other changes to the run command are needed.   As you see the training run is started, you will see the log files show exactly where the code changes are happening to change from GPU to Intel Gaudi, including the file name and location.

Look for the [context] and [hpu_match] in the log file to see where the code is changed.

Remember that if you do not download the dataset the training will not complete the execution, but you will see the GPU Migration changes in the logfile, this is the most important part.

```bash
GPU_MIGRATION_LOG_LEVEL=1 torchrun --nproc_per_node 1 train.py --batch-size=256 --model=resnet50 --device=cuda --data-path=/root/software/data/pytorch/imagenet/ILSVRC2012 --workers=8 --epochs=1 --opt=sgd --amp
```

In [6]:
!GPU_MIGRATION_LOG_LEVEL=1 torchrun --nproc_per_node 1 train.py --batch-size=256 --model=resnet50 --device=cuda --data-path=/root/software/data/pytorch/imagenet/ILSVRC2012 --workers=8 --epochs=1 --opt=sgd --amp

/usr/local/lib/python3.10/dist-packages/habana_frameworks/torch/gpu_migration/__init__.py:46: UserWarning: apex not installed, gpu_migration will not swap api for this package.
  warnings.warn(
gpu migration log will be saved to /var/log/habana_logs/gpu_migration_logs/2024-03-21/00-21-25/gpu_migration_8399.log
[2024-03-21 00:21:25] /root/Gaudi-tutorials/PyTorch/Single_card_tutorials/Model-References/PyTorch/examples/gpu_migration/computer_vision/classification/torchvision/utils.py:265
    [context]:     torch.cuda.set_device(args.gpu)

    [hpu_match]: torch.cuda.set_device(device=0, ) --> torch.hpu.set_device(hpu:0)

| distributed init (rank 0): env://
[2024-03-21 00:21:27] /root/Gaudi-tutorials/PyTorch/Single_card_tutorials/Model-References/PyTorch/examples/gpu_migration/computer_vision/classification/torchvision/utils.py:269
    [context]:     torch.distributed.init_process_group(

    [hpu_match]: torch.distributed.init_process_group(backend=nccl, init_method=env://, timeout=0:30:0

[2023-06-09 23:17:26] train.py:32
    [context]:         image, target = image.to(device, non_blocking=True), target.to(device, non_blocking=True)

    [hpu_match]: torch.Tensor.to(args=(device(type='cuda'),), kwargs={'non_blocking': True}, ) --> torch.Tensor.to(args=('hpu',), kwargs={non_blocking=True, })

[2023-06-09 23:17:26] train.py:33
    [context]:         with torch.cuda.amp.autocast(enabled=scaler is not None):

    [hpu_match]: torch.autocast.__init__(device_type=cuda, dtype=torch.float16, enabled=True, cache_enabled=True, ) --> torch.autocast.__init__(device_type=hpu, dtype=None, enabled=True, cache_enabled=True, )

[2023-06-09 23:17:26] /usr/local/lib/python3.8/dist-packages/torch/cuda/amp/common.py:7
    [context]:     return not (torch.cuda.is_available() or find_spec('torch_xla'))

    [hpu_match]: torch.cuda.is_available() --> torch.hpu.is_available()

[2023-06-09 23:17:37] /root/tf/Model-References/PyTorch/examples/gpu_migration/computer_vision/classification/torchvisi

Epoch: [0]  [370/622]  eta: 0:00:35  lr: 0.1  img/s: 4708.559641186386  loss: 6.0441 (6.4522)  acc1: 1.9531 (1.2747)  acc5: 6.2500 (4.5539)  time: 0.0542  data: 0.0080  max mem: 16966
Epoch: [0]  [380/622]  eta: 0:00:33  lr: 0.1  img/s: 3969.727520264712  loss: 6.0329 (6.4277)  acc1: 1.9531 (1.3822)  acc5: 6.6406 (4.7676)  time: 0.0568  data: 0.0114  max mem: 16966
Epoch: [0]  [390/622]  eta: 0:00:31  lr: 0.1  img/s: 5295.522353315996  loss: 5.9631 (6.4052)  acc1: 1.9531 (1.4258)  acc5: 7.8125 (5.0781)  time: 0.0538  data: 0.0095  max mem: 16966
Epoch: [0]  [400/622]  eta: 0:00:29  lr: 0.1  img/s: 3096.166358897938  loss: 5.9305 (6.3836)  acc1: 1.9531 (1.4863)  acc5: 8.2031 (5.3163)  time: 0.0628  data: 0.0150  max mem: 16966
Epoch: [0]  [410/622]  eta: 0:00:28  lr: 0.1  img/s: 2408.498940252397  loss: 5.9192 (6.3629)  acc1: 2.7344 (1.5532)  acc5: 9.3750 (5.4874)  time: 0.0918  data: 0.0395  max mem: 16966
Epoch: [0]  [420/622]  eta: 0:00:26  lr: 0.1  img/s: 3030.912933452077  loss: 5.

Training time 0:01:23


In [7]:
# Please be sure to run this exit command to ensure that the resorces running on Intel Gaudi are released 
exit()